In [ ]:
import geopandas as gpd
import osmnx as ox
import pandas as pd

In [ ]:
file_path = "../Data/"
output_file_path = file_path + "Output/"

# Street network in LA county

In [ ]:
G = ox.graph_from_place("Los Angeles County, CA, USA", network_type="drive")

# Street network in LA urban settlement area, dataverse

In [ ]:
# street network for LA urban settlement area
G_graphml = ox.io.load_graphml(file_path + "los_angeles-14.graphml")
fig, ax = ox.plot_graph(G_graphml)

In [ ]:
# California census tract downloaded from census bureau
CA_22tract = gpd.read_file(file_path + "tl_2022_06_tract/tl_2022_06_tract.shp")

In [ ]:
LA_22tract = CA_22tract[CA_22tract["COUNTYFP"] == "037"]

In [ ]:
# Los Angeles census tract used by uber movements
uber_tract = gpd.read_file(file_path + "los_angeles_censustracts.json")

In [ ]:
CA_22tract.crs

In [ ]:
CA_22tract_crs = CA_22tract.to_crs(4326)
CA_22tract_crs.crs

In [ ]:
uber_tract.crs

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf = gdf_nodes.reset_index(drop=False)

In [ ]:
gdf_proj = ox.project_gdf(gdf, to_latlong=True)
gdf_proj["x"] = gdf_proj["geometry"].x
gdf_proj["y"] = gdf_proj["geometry"].y

In [ ]:
gdf_proj.crs

# attach (spatial join) census tract GEOID and Uber Movement Tract Movement ID to all the nodes in LA county

In [ ]:
selected_cols = ["osmid", "y", "x", "highway", "street_count", "ref", "geometry", "GEOID"]

gdf_proj_tract = gpd.sjoin(gdf_proj, CA_22tract_crs, how="left", predicate="within")[selected_cols]
gdf_proj_tract_uber = gpd.sjoin(gdf_proj_tract, uber_tract, how="left", predicate="within")[
    selected_cols + ["MOVEMENT_ID", "TRACT"]
]
gdf_proj_tract_uber

# Sample 100,000 nodes as origin and destination (100,000 OD pairs) in Los Angeles County

In [ ]:
origin = gdf_proj_tract_uber.sample(100000, random_state=12345, replace=True).copy()
origin = origin[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
origin.columns = ["oid", "oy", "ox", "oGEOID", "oMOVEMENT_ID"]
destin = gdf_proj_tract_uber.sample(100000, random_state=12345, replace=True).copy()
destin = destin[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
destin.columns = ["did", "dy", "dx", "dGEOID", "dMOVEMENT_ID"]

In [ ]:
origin = origin.reset_index(drop=True)
destin = destin.reset_index(drop=True)

In [ ]:
temp_OD = pd.concat([origin, destin], sort=False, axis=1)
temp_OD = temp_OD.drop(temp_OD[temp_OD["oid"] == temp_OD["did"]].index)

# attach (spatial join) census tract GEOID and Uber Movement Tract Movement ID to all the nodes in LA urban area

In [ ]:
gdfml_nodes, gdfml_edges = ox.graph_to_gdfs(G_graphml)
gdfml = gdfml_nodes.reset_index(drop=False)

In [ ]:
gdfml_proj = ox.project_gdf(gdfml, to_latlong=True)
gdfml_proj["x"] = gdfml_proj["geometry"].x
gdfml_proj["y"] = gdfml_proj["geometry"].y

In [ ]:
gdfml_proj_tract = gpd.sjoin(gdfml_proj, CA_22tract_crs, how="left", predicate="within")
gdfml_proj_tract_clean = gdfml_proj_tract[
    ["osmid", "y", "x", "highway", "street_count", "ref", "geometry", "GEOID"]
]
gdfml_proj_tract_uber = gpd.sjoin(
    gdfml_proj_tract_clean,
    uber_tract,
    how="left",
    predicate="within",
)
gdfml_proj_tract_uber_clean = gdfml_proj_tract_uber[
    [
        "osmid",
        "y",
        "x",
        "highway",
        "street_count",
        "ref",
        "geometry",
        "GEOID",
        "MOVEMENT_ID",
        "TRACT",
    ]
]
gdfml_proj_tract_uber_clean

# Sample 100,000 nodes as origin and destination (100,000 OD pairs) in Los Angeles Urban Area

In [ ]:
originml = gdfml_proj_tract_uber_clean.sample(100000, replace=True).copy()
originml = originml[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
originml.columns = ["oid", "oy", "ox", "oGEOID", "oMOVEMENT_ID"]
destinml = gdfml_proj_tract_uber_clean.sample(100000, replace=True).copy()
destinml = destinml[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
destinml.columns = ["did", "dy", "dx", "dGEOID", "dMOVEMENT_ID"]

In [ ]:
originml = originml.reset_index(drop=True)
destinml = destinml.reset_index(drop=True)

In [ ]:
tempml_OD = pd.concat([originml, destinml], sort=False, axis=1)
tempml_OD = tempml_OD.drop(tempml_OD[tempml_OD["oid"] == tempml_OD["did"]].index)

# Sampling OD pairs that are in both the Los Angeles Urban Area and Los Angeles county

In [ ]:
la_county_ml_intersect = gdfml_proj_tract_uber_clean[~gdfml_proj_tract_uber_clean["GEOID"].isna()]

In [ ]:
la_county_ml_intersect = la_county_ml_intersect.loc[
    la_county_ml_intersect["GEOID"].str.startswith("06037")
]

In [ ]:
# percentage of nodes in LA urban area that are also in LA county
len(la_county_ml_intersect) / len(gdfml_proj_tract_uber_clean)

In [ ]:
ax = LA_22tract.plot(color="palegreen", edgecolor="green", figsize=(20, 10))
gdfml_proj_tract_uber_clean.plot(ax=ax, color="red")

In [ ]:
origin_intersect = la_county_ml_intersect.sample(100000, replace=True).copy()
origin_intersect = origin_intersect[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
origin_intersect.columns = ["oid", "oy", "ox", "oGEOID", "oMOVEMENT_ID"]
destin_intersect = la_county_ml_intersect.sample(100000, replace=True).copy()
destin_intersect = destin_intersect[["osmid", "y", "x", "GEOID", "MOVEMENT_ID"]]
destin_intersect.columns = ["did", "dy", "dx", "dGEOID", "dMOVEMENT_ID"]

In [ ]:
origin_intersect

In [ ]:
origin_intersect = origin_intersect.reset_index(drop=True)
destin_intersect = destin_intersect.reset_index(drop=True)
temp_intersect_OD = pd.concat([origin_intersect, destin_intersect], sort=False, axis=1)
temp_intersect_OD = temp_intersect_OD.drop(
    temp_intersect_OD[temp_intersect_OD["oid"] == temp_intersect_OD["did"]].index,
)

# Save sampled PD pairs as csv

In [ ]:
temp_OD.to_csv(output_file_path + "temp_OD.csv")
tempml_OD.to_csv(output_file_path + "tempml_OD.csv")
temp_intersect_OD.to_csv(output_file_path + "temp_intersect_OD.csv")